# 롯데리아 상품권 잔액 0 만들기
> 이하 내용은 100% 사실에 기반하고 있습니다.

L사 면접에서 면접비 대신 30,000원짜리 상품권을 받았다. 해당 상품권은 L사의 브랜드 식당, 패스트푸드점, 도넛 가게, 카페 등에서만 사용이 가능했다. 그러던 도중 코로나로 인한 거리두기가 완화되어 스터디 카페에 나가 공부하게 됐다. 돈을 아끼기 위해 상품권으로 패스트푸드점에서 점심을 해결하기로 했다. 몇 끼를 어떤 메뉴의 조합으로 때울 수 있을지 고민이 된다. 이왕이면 괜찮은 메뉴, 좋아하는 메뉴가 들어간 조합이었으면 좋겠고, 더해서 상품권의 잔액이 깔끔하게 1원도 남지 않았으면 좋겠다. <br>

아래는 그런 문제를 해결하기 위한 진행과정+코드이다.

## 롯데리아 버거 메뉴+가격 크롤링
우선은 롯데리아에 있는 모든 버거 메뉴와 그 가격을 알아야 한다. BeautifulSoup을 이용한 웹 크롤링으로 버거 이름과 가격을 긁어온다.

In [1]:
# 가격 크롤링 코드
# 크롤링할 주소 1 : http://www.lotteria.com/menu/MenuList.asp?C2=4(햄버거)
# 크롤링할 주소 2 : http://www.lotteria.com/menu/MenuList.asp?C2=6(사이드메뉴)
# 태그 class : cont menu roundMiddle > strong 안의 text
# 브랜드 사이트와 내가 갖고 있는 돈 입력하면 최적의 조합 알려주기(아이디어)

import requests
from bs4 import BeautifulSoup 
import time
import re

url='http://www.lotteria.com/menu/MenuList.asp?C2=4'
name_li=[]
price_li=[]

web=requests.get(url).content
source=BeautifulSoup(web, 'html.parser')
        
for name in source.select('.roundMiddle > a'):
    name_li.append(name.get_text())
    
for price in source.select('.roundMiddle > strong'):
    price_li.append(price.get_text())
        
# print('버거 이름 리스트 :', name_li)
# print('가격 리스트 :', price_li)

# menu_dict=dict(zip(name_li, price_li))
# print(menu_dict)

## 버거 메뉴와 가격 dict로 만들고 정렬하기

In [2]:
price_li2=[]

for price in price_li:
    price_li2.append(int(price.strip('원').replace(',', '')))
    
menu_dict=dict(zip(name_li, price_li2))
print(menu_dict)

{'와규 에디션Ⅱ': 6000, '더블X2': 5700, '모짜렐라 인 더 버거 - 베이컨': 6200, 'AZ버거': 6800, '원조 빅불': 5900, '핫크리스피버거': 4900, '한우불고기': 7200, '불고기버거': 3900, '폴더버거 핫치킨': 5800, '폴더버거 비프': 5800, '리아미라클버거': 5700, '사각새우더블버거': 4500, '새우버거': 3900, '치킨버거': 3100, 'T-Rex': 3800, '클래식 치즈버거': 4500, '데리버거': 2700}


In [3]:
menu_dict=sorted(menu_dict.items(), key=lambda x:x[1], reverse=True)
print(menu_dict) # 자료형이 dict-> list로 자동 변환됨

[('한우불고기', 7200), ('AZ버거', 6800), ('모짜렐라 인 더 버거 - 베이컨', 6200), ('와규 에디션Ⅱ', 6000), ('원조 빅불', 5900), ('폴더버거 핫치킨', 5800), ('폴더버거 비프', 5800), ('더블X2', 5700), ('리아미라클버거', 5700), ('핫크리스피버거', 4900), ('사각새우더블버거', 4500), ('클래식 치즈버거', 4500), ('불고기버거', 3900), ('새우버거', 3900), ('T-Rex', 3800), ('치킨버거', 3100), ('데리버거', 2700)]


In [4]:
menu_dict=dict(map(reversed,menu_dict))
print(menu_dict)

{7200: '한우불고기', 6800: 'AZ버거', 6200: '모짜렐라 인 더 버거 - 베이컨', 6000: '와규 에디션Ⅱ', 5900: '원조 빅불', 5800: '폴더버거 비프', 5700: '리아미라클버거', 4900: '핫크리스피버거', 4500: '클래식 치즈버거', 3900: '새우버거', 3800: 'T-Rex', 3100: '치킨버거', 2700: '데리버거'}


## 버거 가격들만 리스트로 만들기

In [14]:
burger_li=list(menu_dict.keys())
print(burger_li)

[7200, 6800, 6200, 6000, 5900, 5800, 5700, 4900, 4500, 3900, 3800, 3100, 2700]


## itertools의 combinations_with_replacement 활용해 결과값 찾기 
버거 주문은 중복이 가능하므로 중복을 허용하는 경우의 수를 뽑아내는 combinations_with_replacement를 사용한다. 

In [31]:
from itertools import combinations_with_replacement

result=[]

com_li=list(combinations_with_replacement(burger_li, 4))

for com in com_li:
    if sum(com)==17600:
        t=tuple(menu_dict.get(com[i]) for i in range(4)) # 각 숫자를 key로 버거 이름 value를 추출해 tuple에 담는다
        result.append(t)

In [32]:
result

[('한우불고기', '새우버거', 'T-Rex', '데리버거'),
 ('AZ버거', '새우버거', 'T-Rex', '치킨버거'),
 ('모짜렐라 인 더 버거 - 베이컨', '와규 에디션Ⅱ', '데리버거', '데리버거'),
 ('모짜렐라 인 더 버거 - 베이컨', '핫크리스피버거', 'T-Rex', '데리버거'),
 ('모짜렐라 인 더 버거 - 베이컨', '클래식 치즈버거', 'T-Rex', '치킨버거'),
 ('모짜렐라 인 더 버거 - 베이컨', 'T-Rex', 'T-Rex', 'T-Rex'),
 ('와규 에디션Ⅱ', '폴더버거 비프', '치킨버거', '데리버거'),
 ('와규 에디션Ⅱ', '새우버거', '새우버거', 'T-Rex'),
 ('원조 빅불', '원조 빅불', '치킨버거', '데리버거'),
 ('원조 빅불', '클래식 치즈버거', '클래식 치즈버거', '데리버거'),
 ('원조 빅불', '새우버거', '새우버거', '새우버거'),
 ('폴더버거 비프', '핫크리스피버거', 'T-Rex', '치킨버거'),
 ('리아미라클버거', '리아미라클버거', '치킨버거', '치킨버거'),
 ('리아미라클버거', '핫크리스피버거', '새우버거', '치킨버거'),
 ('핫크리스피버거', '핫크리스피버거', '새우버거', '새우버거')]

## 함수로 만들기

In [39]:
import requests
from bs4 import BeautifulSoup 
import time
from itertools import combinations_with_replacement

def saveMyMoney(money, days):
    result=[]

    com_li=list(combinations_with_replacement(burger_li, days))
    for com in com_li:
        if sum(com)==money:
            t=tuple(menu_dict.get(com[i]) for i in range(days)) # 각 숫자를 key로 버거 이름 value를 추출해 tuple에 담는다
            result.append(t)
   
    return result
    
if __name__ == '__main__':
    m=int(input('상품권 잔액을 입력하세요 :'))
    d=int(input('희망하는 점심 채움 횟수를 입력하세요 :'))

    url='http://www.lotteria.com/menu/MenuList.asp?C2=4'
    name_li=[]
    price_li=[]

    web=requests.get(url).content
    source=BeautifulSoup(web, 'html.parser')

    for name in source.select('.roundMiddle > a'):
        name_li.append(name.get_text())

    for price in source.select('.roundMiddle > strong'):
        price_li.append(price.get_text())
    
    price_li2=[]

    for price in price_li:
        price_li2.append(int(price.strip('원').replace(',', '')))

    menu_dict=dict(zip(name_li, price_li2))
    menu_dict=sorted(menu_dict.items(), key=lambda x:x[1], reverse=True)
    menu_dict=dict(map(reversed,menu_dict))
    burger_li=list(menu_dict.keys())
    
    
    answer=saveMyMoney(m, d)
    for a in answer:
        print(a)

상품권 잔액을 입력하세요 :30000
9
('한우불고기', '새우버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거')
('한우불고기', '치킨버거', '치킨버거', '치킨버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거')
('AZ버거', '새우버거', '치킨버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거')
('AZ버거', '치킨버거', '치킨버거', '치킨버거', '치킨버거', '데리버거', '데리버거', '데리버거', '데리버거')
('모짜렐라 인 더 버거 - 베이컨', '핫크리스피버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거')
('모짜렐라 인 더 버거 - 베이컨', '클래식 치즈버거', '치킨버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거')
('모짜렐라 인 더 버거 - 베이컨', 'T-Rex', 'T-Rex', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거')
('와규 에디션Ⅱ', '새우버거', '새우버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거')
('와규 에디션Ⅱ', '새우버거', '치킨버거', '치킨버거', '치킨버거', '데리버거', '데리버거', '데리버거', '데리버거')
('와규 에디션Ⅱ', '치킨버거', '치킨버거', '치킨버거', '치킨버거', '치킨버거', '치킨버거', '데리버거', '데리버거')
('폴더버거 비프', '핫크리스피버거', '치킨버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거')
('폴더버거 비프', '클래식 치즈버거', '치킨버거', '치킨버거', '데리버거', '데리버거', '데리버거', '데리버거', '데리버거')
('폴더버거 비프', 'T-Rex', 'T-Rex',

In [ ]:
# 알고리즘 고민 중
import random

length=len(int_li)
total_price=17600
result=[]

for price in int_li:
    max_price=[price]
    print(max_price)
    for i in range(length):
        price_case=max_price+[int_li[i]]
        print(price_case)
        if sum(price_case)==total_price:
            result.append(price_case)
            
print(result)